# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
def grayscale(img, weights=np.array([0.299, 0.587, 0.114])):
    """
    Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')
    """
    return np.uint8(np.average(img, weights=weights, axis=2))
    # return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).

    Think about things like separating line segments by their
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of
    the lines and extrapolate to the top and bottom of the lane.

    This function draws `lines` with `color` and `thickness`.
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    if lines is None:
        return img

    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def find_hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
    Retunrs list of hough lines
    """
    return cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), 
                           minLineLength=min_line_len, 
                           maxLineGap=max_line_gap)
    
def draw_hough_lines(img, lines, thickness):
    """
    Returns an image with hough lines drawn.
    """
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines, thickness=2)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
"""
Helper function to calculate slopes
"""
def slope(x1, y1, x2, y2):
    return float('Inf') if x2 - x1 == 0 else (y2 - y1) / (x2 - x1)

"""
Helper function to calculate second point for the line with given slope
"""
def line(x0, y0, slope, y):
    return int((y - y0) / slope + x0)

In [ ]:
"""Helper classes to dump images during process"""
class DummyDumper:
    def dump(self, image, stage):
        pass
    
class FileDumper:
    def __init__(self, name, destination):
        self._name, self._ext = os.path.splitext(os.path.basename(name))
        self._destination = destination
        os.makedirs(self._destination, exist_ok=True)
        
    def dump(self, image, stage):
        path = os.path.join(self._destination, self._name + "." + stage + self._ext)
        cv2.imwrite(path, image)

In [ ]:
"""
Helper class for smooth lane updates
"""
class Averager:
    def __init__(self, shape, step):
        self._shape = shape
        self._step = step
        self._bottom = None # x,y coordinate
        self._top = None # x,y coordinate
        
    def update(self, top, slope):
        if top is None or slope is None:
            return
        
        new_top = np.array(top, dtype=np.int64)
        new_bottom = np.array(
            (max(0, min(self._shape[0], line(*top, slope, self._shape[1]))), self._shape[1]), 
            dtype=np.int64)
        
        if self._bottom is None or self._top is None:
            self._top = new_top
            self._bottom = new_bottom
        else:
            self._top += np.int64((new_top - self._top) * self._step)
            self._bottom += np.int64((new_bottom - self._bottom) * self._step)
        
    def propose(self):
        return (self._bottom, self._top)



In [ ]:
"""Calculate region of interest"""
def roi(image, outer_base, inner_base, center, top, bottom):
    
    def roi_width(base, top, center):
        return ((1. - 2 * base * (top - center)) / 2., (1. - base) / 2.)
    
    def roi_shape(shape, top, bottom, o_top, o_bottom, i_top, i_bottom):
        outer_top = shape[1] * o_top
        inner_top = shape[1] * i_top
        top = shape[0] * top
        outer_bottom = shape[1] * o_bottom
        inner_bottom = shape[1] * i_bottom
        bottom = shape[0] * bottom 
        
        return np.array([[outer_bottom, bottom], 
                         [outer_top, top], 
                         [inner_top, top], 
                         [inner_bottom, bottom]],
                        dtype=np.int32)
    
    def roi_slope(shape):
        return sorted([slope(shape[0,0], shape[0,1], shape[1, 0], shape[1,1]),
                       slope(shape[2,0], shape[2,1], shape[3, 0], shape[3,1])])
    
    outer_top, outer_bottom = roi_width(outer_base, top, center)
    inner_top, inner_bottom = roi_width(inner_base, top, center)

    roi_left = roi_shape(image.shape, top, bottom, outer_top, outer_bottom, inner_top, inner_bottom)
    roi_left_slope = roi_slope(roi_left)
    roi_left_image = region_of_interest(image, [roi_left])

    roi_right = roi_shape(image.shape, top, bottom, 1-outer_top, 1-outer_bottom, 1-inner_top, 1-inner_bottom)
    roi_right_slope = roi_slope(roi_right)
    roi_right_image = region_of_interest(image, [roi_right])
    
    return (cv2.bitwise_or(roi_left_image, roi_right_image), 
            np.array([roi_left_slope, roi_right_slope]))

In [ ]:
def analyze_hough_lines(img, lines, slopes):
    def split_hough_lines(img, lines, slopes):
        left = []
        right = []
        for line in lines:
            for x1, y1, x2, y2 in line:
                s = slope(x1, y1, x2, y2)
                # segments extra filtering (belogns to the right half of the image and in right direction)
                if all((s * a < 0 or s < a or s > b for (a, b) in slopes)): continue
                if s >= 0 and min(x1, x2) < img.shape[1] / 2: continue
                if s < 0 and max(x1, x2) > img.shape[1] / 2: continue
                # sorting out which lane we're processing now
                right.append((x1,y1,x2,y2,s)) if s >= 0 else left.append((x1,y1,x2,y2,s))
                #cv2.line(img,(x1,y1),(x2,y2),[128,128,0] if s >=0 else [128,0,128], 2)
        return np.array(left), np.array(right)
    
    def mean_line(lines):
        if lines is None or len(lines) == 0:
            return ((None, None), None)
        
        m = np.mean(lines[:,4])
        # searching for a segement with approximately same slope as median
        argmin = np.argmin(np.power(lines[:,4] - m, 2))
        # taking the middle of the segment
        x = (lines[argmin,0] + lines[argmin, 2]) / 2.
        y = (lines[argmin,1] + lines[argmin, 3]) / 2.
        # searching for highest y value
        argmin = np.argmin(lines, axis=0)
        argidx = 1 if lines[argmin[1], 1] < lines[argmin[3], 3] else 3
        miny = lines[argmin[argidx], argidx]
        # evaluate x coordinate for highest y value using x, y and slope for the best line
        minx = line(x, y, m, miny)
        return ((minx, miny), m)
    
    if lines is None:
        return ((None, None), (None, None))
    
    left, right = split_hough_lines(img, lines, slopes)
    return (mean_line(left), mean_line(right))

In [ ]:
def pipeline(image, left_control, right_control, dumper=DummyDumper()):

    weights = np.array([1.25, 0.25, -0.5]) #0.7, 0.7, -0.4 #1.4 0.5 -1
    gray_image = grayscale(image, weights)
    dumper.dump(gray_image, "gray")
    
    blur_image = gaussian_blur(gray_image, 7)
    dumper.dump(blur_image, "blur")
        
    canny_low = 120 #120 #300
    canny_high = 350 #350 #600
    canny_image = canny(gray_image, canny_low, canny_high)
    dumper.dump(canny_image, "canny")
    
    roi_outer_base = 0.97 #0.98 # fraction of width
    roi_inner_base = 0.33 #0.28 # fraction of width
    roi_center = 0.556 #0.558 # vertical perspective center
    roi_height = 0.608 #0.558 # fraction of height
    roi_bottom = 0.93
    canny_roi_image, slopes = roi(canny_image, roi_outer_base, roi_inner_base, roi_center, roi_height, roi_bottom)
    dumper.dump(canny_roi_image, "roi")
    
    rho = 2
    theta = math.pi / 180
    threshold = 25
    min_line_len = 30
    max_line_gap = 30
    hough_lines = find_hough_lines(canny_roi_image, rho, theta, threshold, min_line_len, max_line_gap)
    #hough_image = draw_hough_lines(canny_roi_image, hough_lines, thickness=1)
    #dumper.dump(hough_image, "hough")
    
    (ltop, lslope), (rtop, rslope) = analyze_hough_lines(canny_roi_image, hough_lines, slopes)
        
    left_control.update(ltop, lslope)
    right_control.update(rtop, rslope)

    lanes_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    (x0,y0),(x1,y1) = left_control.propose()
    cv2.line(lanes_image, (x0,y0), (x1, y1), [255,0,0], 4)

    (x0,y0),(x1,y1) = right_control.propose()
    cv2.line(lanes_image, (x0,y0), (x1, y1), [255,0,0], 4)
    dumper.dump(lanes_image, "lanes")

    return weighted_img(lanes_image, image, α=0.9, β=1., λ=0.)

In [ ]:
import glob

source = "test_images/"
for f in glob.glob(source + "*.jpg"):
    img = mpimg.imread(f)
    left = Averager((img.shape[1],img.shape[0]), 1)
    right = Averager((img.shape[1],img.shape[0]), 1)
    dumper = FileDumper(f, "test_images_output")
    
    pipeline(img, left, right, dumper)

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
"""Helper class to pass Averager into imagre processing chain"""
class ImageProcessor:
    def __init__(self, shape, step = 1./7):
        self._step = step
        self._left = Averager(shape, step)
        self._right = Averager(shape, step)
        
    def process_image(self, image):
        return pipeline(image, self._left, self._right)

Let's try the one with the solid white lane on the right first ...

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
ip = ImageProcessor(clip1.size)
white_clip = clip1.fl_image(ip.process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
ip = ImageProcessor(clip2.size)
yellow_clip = clip2.fl_image(ip.process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
ip = ImageProcessor(clip3.size)
challenge_clip = clip3.fl_image(ip.process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))